In [1]:
import zipfile

with zipfile.ZipFile('datasets/celeba-hq/img_align_celeba.zip', 'r') as zip_ref:
    zip_ref.extractall('datasets/celeba-hq/raw_images')


KeyboardInterrupt: 

In [ ]:
from PIL import Image
import os

input_dir = 'datasets/celeba-hq/raw_images'
output_dir = 'datasets/celeba-hq/val_source_256'
os.makedirs(output_dir, exist_ok=True)

for fname in os.listdir(input_dir):
    if fname.endswith('.jpg'):
        img = Image.open(os.path.join(input_dir, fname)).convert('RGB')
        img = img.resize((256, 256), Image.LANCZOS)
        img.save(os.path.join(output_dir, fname))


In [3]:
import os

img_dir = "datasets/celeba-hq/val_source_256"
mask_dir = "datasets/celeba-hq/val_masks_thick"

images = sorted(os.listdir(img_dir))
masks = sorted(os.listdir(mask_dir))

print(f"Number of images: {len(images)}")
print(f"Number of masks: {len(masks)}")


Number of images: 202599
Number of masks: 93434


In [2]:
import os
from sklearn.model_selection import train_test_split

# Define dataset paths
dataset_root = 'datasets/celeba-hq'
image_dir = os.path.join(dataset_root, 'val_source_256')
mask_dir = os.path.join(dataset_root, 'val_masks_thick')

# List image and mask files
images = sorted(os.listdir(image_dir))
masks = sorted(os.listdir(mask_dir))

# Split into train/validation (90% train, 10% val)
train_imgs, val_imgs, train_masks, val_masks = train_test_split(
    images, masks, test_size=0.1, random_state=42
)

# Function to save lists into files
def save_flist(file_list, file_path):
    with open(file_path, 'w') as file:
        for item in file_list:
            file.write(f"{item}\n")

# Create directory for flist files if it doesn't exist
flist_dir = os.path.join(dataset_root, 'flists')
os.makedirs(flist_dir, exist_ok=True)

# Save the flists
save_flist(train_imgs, os.path.join(flist_dir, 'train_images.flist'))
save_flist(train_masks, os.path.join(flist_dir, 'train_masks.flist'))
save_flist(val_imgs, os.path.join(flist_dir, 'val_images.flist'))
save_flist(val_masks, os.path.join(flist_dir, 'val_masks.flist'))

print("File lists (.flist) generated successfully in:", flist_dir)


ValueError: Found input variables with inconsistent numbers of samples: [202599, 93434]